# Runaway Automation: The $50,000 AWS Bill

## Overview

- **Domain**: DevOps & Cloud Infrastructure
- **Risk Level**: Critical
- **Time**: 10-15 minutes
- **Prevents**: Runaway costs, infinite loops, budget exhaustion

## Real-World Context

A single bug in an automation script can cause catastrophic financial damage. This actually happened to multiple startups: a DevOps agent with a bug enters an infinite loop, spinning up thousands of cloud instances over a weekend. Monday morning brings a five-figure AWS bill that could bankrupt the company.

## What You'll Learn

- How bugs in agent code can cause runaway spending
- Why rate limiting is critical for cloud operations
- How time-based policies prevent infinite loops
- Cost control strategies for agent systems

In [1]:
import sys
sys.path.insert(0, '../')

import requests
import json
from datetime import datetime

# Verify Relay is running
try:
    requests.get('http://localhost:8181/health', timeout=2)
    print('✅ Relay infrastructure is running')
except:
    print('❌ Start infrastructure: cd infra && docker-compose up -d')
    raise

✅ Relay infrastructure is running


## The Scenario

Your DevOps agent has a bug in its auto-scaling logic. The `needs_scaling()` function always returns `True`, causing it to spin up expensive EC2 instances in an infinite loop.

In [2]:
buggy_code = '''
# Buggy agent code
while True:
    if needs_scaling():
        create_ec2_instance(type="m5.8xlarge")  # $1.536/hour
    
    # BUG: needs_scaling() always returns True!
    # Missing break condition
    # No rate limiting
'''

print("🐛 Buggy Agent Code:")
print(buggy_code)

🐛 Buggy Agent Code:

# Buggy agent code
while True:
    if needs_scaling():
        create_ec2_instance(type="m5.8xlarge")  # $1.536/hour

    # BUG: needs_scaling() always returns True!
    # Missing break condition
    # No rate limiting



## Without Relay: Financial Disaster

The bug runs unchecked over the weekend.

In [3]:
print("❌ WITHOUT RELAY:")
print("─" * 80)

print("\n💸 Disaster Unfolds (Friday 6pm - Monday 9am):")
timeline = [
    ("Hour 1", 60, 92),
    ("Hour 2", 120, 184),
    ("Hour 3", 240, 368),
    ("Hour 6", 960, 1475),
    ("Hour 12", 3840, 5898),
    ("Hour 24", 15360, 23593)
]

for time, instances, cost in timeline:
    print(f"   {time:8s}: {instances:5d} instances (${cost:,}/hour)")

print("\n   ...")
print("   Hour 63: AWS bill reaches $50,000+")
print("\n📧 Monday morning:")
print("   └─ AWS billing alert")
print("   └─ Startup nearly bankrupted")
print("   └─ Emergency all-hands to kill instances")
print("   └─ Damage already done")

❌ WITHOUT RELAY:
────────────────────────────────────────────────────────────────────────────────

💸 Disaster Unfolds (Friday 6pm - Monday 9am):
   Hour 1  :    60 instances ($92/hour)
   Hour 2  :   120 instances ($184/hour)
   Hour 3  :   240 instances ($368/hour)
   Hour 6  :   960 instances ($1,475/hour)
   Hour 12 :  3840 instances ($5,898/hour)
   Hour 24 : 15360 instances ($23,593/hour)

   ...
   Hour 63: AWS bill reaches $50,000+

📧 Monday morning:
   └─ AWS billing alert
   └─ Startup nearly bankrupted
   └─ Emergency all-hands to kill instances
   └─ Damage already done


## With Relay: Rate Limiting Saves the Day

Relay enforces hourly spending limits that prevent runaway costs.

In [4]:
print("✅ WITH RELAY:")
print("─" * 80)

print("\n🛡️  Policy: Maximum $100 cloud spending per hour")
print("\n🤖 Agent Attempts (simulated):")

# Simulate multiple instance creation attempts
# Each m5.8xlarge costs ~$25 in our simplified model
attempts = [
    {"instance": 1, "cost": 25, "cumulative": 25},
    {"instance": 2, "cost": 25, "cumulative": 50},
    {"instance": 3, "cost": 25, "cumulative": 75},
    {"instance": 4, "cost": 25, "cumulative": 100},
    {"instance": 5, "cost": 25, "cumulative": 125},
]

for attempt in attempts:
    if attempt["cumulative"] <= 100:
        print(f"   ✅ Instance {attempt['instance']}: "
              f"APPROVED (${attempt['cumulative']} cumulative)")
    else:
        print(f"   🚫 Instance {attempt['instance']}: BLOCKED")
        print(f"      └─ Would exceed hourly limit (${attempt['cumulative']} > $100)")
        print(f"      └─ Agent must wait until next hour")
        print(f"      └─ Alert sent to DevOps team")
        break

print("\n💡 Result:")
print("   ✅ Maximum damage: $100/hour")
print("   ✅ Bug discovered within the hour")
print("   ✅ Total cost: ~$100 (vs $50,000)")
print("   ✅ Startup saved from bankruptcy")

✅ WITH RELAY:
────────────────────────────────────────────────────────────────────────────────

🛡️  Policy: Maximum $100 cloud spending per hour

🤖 Agent Attempts (simulated):
   ✅ Instance 1: APPROVED ($25 cumulative)
   ✅ Instance 2: APPROVED ($50 cumulative)
   ✅ Instance 3: APPROVED ($75 cumulative)
   ✅ Instance 4: APPROVED ($100 cumulative)
   🚫 Instance 5: BLOCKED
      └─ Would exceed hourly limit ($125 > $100)
      └─ Agent must wait until next hour
      └─ Alert sent to DevOps team

💡 Result:
   ✅ Maximum damage: $100/hour
   ✅ Bug discovered within the hour
   ✅ Total cost: ~$100 (vs $50,000)
   ✅ Startup saved from bankruptcy


## Cost Control Analysis

Compare the financial impact with and without Relay.

In [5]:
cost_comparison = {
    "without_relay": {
        "weekend_cost": 50000,
        "instances_created": 15360,
        "detection_time": "63 hours",
        "business_impact": "Critical - nearly bankrupted"
    },
    "with_relay": {
        "weekend_cost": 100,
        "instances_created": 4,
        "detection_time": "< 1 hour",
        "business_impact": "Minimal - bug fixed quickly"
    },
    "savings": 49900,
    "roi": "49,800%"
}

print("📊 Cost Analysis:")
print(json.dumps(cost_comparison, indent=2))

📊 Cost Analysis:
{
  "without_relay": {
    "weekend_cost": 50000,
    "instances_created": 15360,
    "detection_time": "63 hours",
    "business_impact": "Critical - nearly bankrupted"
  },
  "with_relay": {
    "weekend_cost": 100,
    "instances_created": 4,
    "detection_time": "< 1 hour",
    "business_impact": "Minimal - bug fixed quickly"
  },
  "savings": 49900,
  "roi": "49,800%"
}


## Try It With Curl

Test rate limiting without any installation:

```bash
# Simulate rapid instance creation attempts
# First 4 succeed, 5th is blocked

for i in {1..5}; do
  echo "\nAttempt $i:"
  curl -X POST http://localhost:8181/v1/data/relay/policies/main \
    -H "Content-Type: application/json" \
    -d '{
      "input": {
        "action": {
          "provider": "aws",
          "method": "create_instance",
          "parameters": {
            "instance_type": "m5.8xlarge",
            "cost_estimate": 2500
          }
        }
      }
    }'
  sleep 1
done
```

Note: Rate limiting implementation depends on your policy configuration.

## Policy Implementation

Here's the policy that prevents this scenario:

```yaml
policies:
  - name: "cloud_spending_rate_limits"
    description: "Prevent runaway cloud costs"
    rules:
      - id: "hourly_spending_limit"
        condition:
          provider: "aws"
          method: "create_instance"
          # Check cumulative spending in last 60 minutes
          rate_limit:
            window: "1h"
            max_cost: 10000  # $100 in cents
        action: "allow"
        reason: "Within hourly spending limit"
      
      - id: "deny_excessive_hourly_spending"
        condition:
          provider: "aws"
          method: "create_instance"
        action: "deny"
        reason: "Hourly spending limit exceeded - possible runaway automation"
        alert:
          channels: ["pagerduty", "slack"]
          severity: "high"
          message: "Agent hit hourly spending limit - investigate for bugs"
```

**Advanced rate limiting strategies:**

1. **Time windows**: Hour, day, week, month
2. **Resource types**: Different limits for different instance types
3. **Per-agent limits**: Junior agents get lower limits
4. **Circuit breakers**: Automatic pause after repeated violations

**To implement this policy:**

1. Add to `/policies/cloud_ops.yaml`
2. Configure cost estimation rules
3. Set up alerting integrations
4. Run: `python scripts/bootstrap_policies.py`
5. Restart Relay gateway
6. Test with gradual rollout

## Key Takeaways

- **Bugs happen** - even in production code from experienced engineers
- **Rate limiting is mandatory** for any system with financial impact
- **Early detection saves money** - hourly limits catch problems before weekends
- **Automation needs guardrails** - especially for cloud resource creation
- **One incident pays for Relay forever** - $50k saved vs $100/month cost

## Next Steps

- **Explore**: [09_gdpr_compliance.ipynb](09_gdpr_compliance.ipynb) - Learn about data protection
- **Learn**: [04_company_policies.ipynb](04_company_policies.ipynb) - Rate limiting policy patterns
- **Review**: [05_real_world_scenarios.ipynb](05_real_world_scenarios.ipynb) - Cost-benefit analysis